#### Autori: Domenico Lembo, Antonella Poggi, Giuseppe Santucci and Marco Schaerf

[Dipartimento di Ingegneria informatica, automatica e gestionale](https://www.diag.uniroma1.it)

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.eu.png"
     alt="License"
     style="float: left;"
     height="40" width="100" />
This notebook is distributed with license Creative Commons *CC BY-NC-SA*

# Espressioni Regolari

1. Cenni sulla teoria delle espressioni regolari
2. Differenze fondamentali tra la teoria delle espressioni regolari e il loro uso in Python
4. Esempi di uso delle espressioni regolari
5. Metodi fondamentali per la ricerca
6. Caratteri denotanti gruppi
7. Ricerca di stringhe con ripetizioni
8. Esempi di esercizi semplici
9. Funzione per la sostituzione
10. Esempi di esercizi più complessi
11. Esercizi

## Cenni sulla teoria delle espressioni regolari 
La teoria delle espressioni regolari è un modello matematico in cui si definiscono formalmente:
- il concetto di stringa
- la sintassi delle espressioni regolari
- la loro semantica

in modo da poter definire un insieme (non necessariamente finito) di stringhe che soddisfano certi requisiti sintattici, per esempio, **tutte le stringhe che cominciano per 'a' e finiscono per 'o'**= {'ao', 'atto', 'axo', 'aao', ...}.
Una stringa è definita su una sequenza (ovvero un insieme **ordinato**) di simboli A detto *alfabeto*, e An denota l'insieme di tutte le stringhe di lunghezza n costruite usando simboli in A mentre A\* denota l'insieme di tutte le stringhe di qualunque lunghezza costruite usando i simboli in A. 

Un'espressione regolare è il modo per specificare un sottoinsieme (non necessariamente proprio) di A\* composto da tutte e sole le stringhe che hanno una certa *forma*, ovvero che rispettano un insieme di vincoli. Si dice che le stringhe che soddisfano tali vincoli **collimano** con l'espressione regolare, quelle che non le soddisfano **non collimano**. Per esempio, 'alto' collima con **tutte le stringhe che cominciano per 'a' e finiscono per 'o'** mentre 'alta' non collima.

La sintassi delle espressioni regolari utilizza i caratteri di A, operatori (e.g., concatenazione, ripetizione, e negazione), parentesi che indicano l'ordine di esecuzione degli operatori, caratteri speciali che indicano specifiche sequenze di caratteri, come: la stringa vuota, una stringa costituita da un unico carattere, una stringa qualsiasi in A\*, una stringa qualsiasi in A1, una stringa costituita da n caratteri compresi in un certo intervallo, ecc.

Per esempio, sapendo che **"."** (Punto, in inglese "dot") corrisponde a qualunque carattere, (\n escluso) e che **"\*"** corrisponde a 0 o più ripetizioni del carattere che lo precede, l'espressione regolare che rappresenta **tutte le stringhe che cominciano per 'a' e finiscono per 'o'** vale: 'a.\*o'.
Primo esempio:


In [1]:
import re
s = 'piedistallo'
m=re.search('a.*o',s)
print(m)

<re.Match object; span=(7, 11), match='allo'>


**Osservazioni sull'esempio sopra:** per usare le espressioni regolari, abbiamo importato il modulo `re` e invocato la funzione `search` del modulo, che prende in input l'espressione regolare e la stringa all'interno della quale cercare le occorrenze delle stringhe che la soddisfano, e restituisce un oggetto di tipo `Match` (cf. sotto).

## Differenze fondamentali tra la teoria delle espressioni regolari e il loro uso in Python
Nella teoria, le espressioni regolari sono un modo per definire insiemi di stringhe in base alle loro proprietà, mentre in Python (come anche in tanti altri linguaggi di programmazione) le espressioni regolari sono più frequentemente usate per **cercare** all'interno di un testo **t** (stringa) una sottostringa **s** (definita con un'espressione regolare che in Python viene spesso chiamata *pattern*) che abbia particolari proprietà. Questo utilizzo introduce alcune complicazioni che siamo costretti ad affrontare, come ad esempio:

- cosa fare quando ci sono più sottostringhe **s** che *collimano* con l'espressione regolare? Troviamo la prima o tutte?

s = 'paapera casa baaviera'

m=re.search('.aa.',s) #trova **'paap'** e/o 'baav' ? (in grassetto il default di Python)

- cosa fare quando ci sono 2 sottostringhe, **s1** e **s2**, che collimano con l'espressione regolare e che si sovrappongono in **t**?


m=re.search('a.*o','allo aot') #trova 'allo' e/o **'allo ao'** ? (in grassetto il default di Python)

- come specificare che la sottostringa da cercare deve essere solo cercata all'inizio (oppure alla fine) della stringa?


m=re.search('ot','ot aot') #trova 'ot' **iniziale** e/o finale ? (in grassetto il default di Python)

Vedremo che Python risolve tutti questi problemi definendo diversi metodi nella classe *re* e un opportuno insieme di caratteri speciali.

### La notazione r'stringa'
Prima di inziare, introduciamo questo modo alternativo di specificare stringhe in Python che è spesso utile nel caso delle espressioni regolari. Se in Python una stringa `s` ha davanti il carattere 'r', allora vuol dire che la stringa è *raw* (cruda, cioè da non interpretare). In questo caso TUTTI i caratteri di `s` rappresentano esattamente se stessi e NON sono caratteri di controllo tipo '\n' o '\t'. Questa notazione è molto utile nelle espressioni regolari che usano frequentemente i caratteri che nelle stringhe Python verrebbero interpretati come caratteri di controllo, come '\\'. Ad esempio:

In [2]:
s = r'\nprova\n'
s1 = '\nprova\n'
print('La stringa s è lunga:',len(s),'e vale:\n',s)
print('La stringa s1 è lunga:',len(s1),'e vale:\n',s1)

La stringa s è lunga: 9 e vale:
 \nprova\n
La stringa s1 è lunga: 7 e vale:
 
prova



**Nota:** Si noti che una stringa raw è una stringa legale Python preceduta da una 'r' (come visto negli esempi precedenti). La sequenza `r"\"` NON è una stringa raw, perchè `"\"` NON è una stringa legale in Python. Lo stesso dicasi per tutte le stringhe che terminano con un numero dispari di `\`. 

## Esempi di uso delle espressioni regolari
Cominciamo con dei semplici esempi di come si possano usare le espressioni regolari in Python. Successivamente vedremo più precisamente le principali operazioni e i caratteri speciali definiti nel modulo *re*.

### Cercare una stringa in un semplice elenco telefonico

Cerchiamo "Mario" in un elenco strutturato così:
elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'

Per ora verifichiamo solo se "Mario" è presente in elenco. Per fare questo useremo la funzione `re.search(..)`.
Questa funzione ha 2 parametri, il primo è il *pattern* (di regola specificato con un'espressione regolare) e il secondo è il testo (stringa) in cui cercare. Notate che la funzione `re.search(..)` restituisce un oggetto di tipo `Match`.

In [3]:
import re

elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'
pattern = 'Mario'

m = re.search(pattern, elenco)
if m:
    print("trovato")
else:
    print("non trovato")

trovato


### Stampare la sottostringa trovata
Il codice precedente stampa solo un messaggio. Con il metodo `group()` applicato al risultato della funzione `re.search` (che è un oggetto di tipo `match`) possiamo stampare la sottostringa che ha *collimato* con l'espressione regolare (pattern).

In [8]:
import re
elenco='Luca Rossi 2345234 - Mario- Bianchi 82342342 - Marco Verdi 342342342'
pattern = 'Mario'

m = re.search(pattern, elenco)
print(m.group())

if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

Mario
trovato: Mario


### Individuare una sequenza di caratteri compresi in un certo insieme
Per individuare una sequenza di caratteri compresi in un certo insieme, usiamo la notazione `[ insieme caratteri ]`, eventualmente seguita da un *indicatore di occorrenza*, ovvero da un simbolo  `*`, `?` o `+`. La notazione [ insieme caratteri ] collima con **una sola occorrenza** di un carattere tra quelli indicati nell'insieme mentre con un indicatore di occorrenza modifichiamo il numero di caratteri che collimano, come segue:

- `?` =  0 o 1 occorrenza 

- `*` =  0 o + occorrenze 

- `+` =  1 o + occorrenze

Supponiamo di volere individuare il numero di telefono di Mario. Dobbiamo scrivere un'espressione regolare per cercare una stringa che non contenga solo il nome ma anche il numero (che non conosciamo, ma sappiamo come è fatto). Cerchiamo quindi una sottostringa che abbia il formato:
- `'Mario'` seguito da un qualunque numero di caratteri alfabetici, spazi o cifre, basta che non ci sia il carattere '-' (che viene usato da separatore nel nostro caso), ovvero `'[a-zA-Z0-9 ]*'`


In [9]:
pattern = 'Mario[a-zA-Z0-9 ]*'  #notare lo spazio dopo il 9
m = re.search(pattern, elenco)
if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

trovato: Mario


### Definizione per negazione
A volte è più semplice definire un'espressione regolare specificando cosa **NON** deve contenere invece di cosa deve contenere. Nel caso precedente, avremmo potuto definire la parte che segue 'Mario' come una sequenza di caratteri **diversi dal carattere '-'**. Il simbolo speciale per denotare la negazione è `^` (attenzione che questo stesso simbolo ha anche il significato di inizio stringa, come vedremo nel seguito), il pattern completo diventa quindi `'Mario[^-]*'`

In [10]:
import re
elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'

pattern = 'Mario[^-]*'

m = re.search(pattern, elenco)
if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

trovato: Mario Bianchi 82342342 


### Individuare le componenti
Molto spesso la sottostringa che cerchiamo è composta da varie parti logicamente divise. Nel nostro esempio, in realtà stiamo cercando una sottostringa composta da 3 parti distinte:
- Il *nome*, che nel nostro caso deve essere obbligatoriamente 'Mario'
- il *cognome*, che non conosciamo, ma assumiamo sia composto da sole lettere maiuscole o minuscole
- il *numero di telefono*, che assumiamo composto da sole cifre.

Queste 3 parti distinte sono separate da uno spazio bianco. Per identificare diverse parti di un'espressione regolare in Python si usano le parentesi tonde. Ogni volta che una parte dell'espressione regolare è racchiusa tra parentesi tonde, Python salva la sottostringa che collima con quella parte e la salva nell'oggetto `Match` restituito dalla `search` come *gruppo* dell'oggetto stesso. I gruppi vengono numerati da `1` in poi. Il gruppo `0` è un gruppo speciale e contiene l'*intera sottostringa* che ha collimato con tutta l'espressione regolare. Per ottenere, a partire da un oggetto di tipo `Match` le stringhe che hanno collimato con le varie parti dell'espressione (o con tutta l'espressione regolare), si usa il metodo `group` dandogli come argomento il numero del gruppo (l'argomento di default è `0`). Vediamo un esempio:

In [16]:
elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'

pattern = '(Mario) (([A-Z])[a-z]*) ([0-9]*)'
m = re.search(pattern, elenco)
if m:
    print("nome: " + m.group(1))
    print("cognome: " + m.group(2))
    print("iniziale del cognome: " + m.group(3))
    print("telefono: " + m.group(4))
else:
    print("non trovato")
print('Gruppo 0:',m.group(0)) # equivalente a: print('Gruppo 0:',m.group())

nome: Mario
cognome: Bianchi
iniziale del cognome: B
telefono: 82342342
Gruppo 0: Mario Bianchi 82342342


**Osservazioni sull'esempio sopra:**
L'espressione regolare `'(Mario) (\[A-Za-z\]\*) (\[0-9\]\*)'` definisce una stringa composta da 3 parti separate da spazio.

(Mario)| (\[A-Za-z\]\*)| (\[0-9\]\*)
:-----:|:---------:|:------:
m.group(1)| m.group(2)|m.group(3)
              
Invece `m.group(0)` è tutta la sottostringa (uguale a `m.group()`).

Notate che gli spazi bianchi sono caratteri come gli altri, se inserite uno spazio nell'espressione regolare questo collima con uno spazio nel testo. Nel nostro caso, infatti, il pattern `'(Mario) (\[A-Za-z\]\*) (\[0-9\]\*)'` contiene uno spazio tra il nome ed il cognome e uno tra il cognome ed il numero di telefono.

### Trovare più sottostringhe diverse
In molti casi non vogliamo trovare una sola sottostringa che collima con la nostra espressione regolare, ma TUTTE le sottostringhe del testo che collimano con il nostro pattern. In questo caso si può usare la funzione `finditer` che esegue più *search* "consumando la stringa" e che restituisce un oggetto iterabile di `Match` ordinati, dove ogni `Match` contiene il risultato di una singola `search`. Vediamo un esempio in cui cerchiamo nel nostro elenco tutte le persone il cui nome inizia per 'M':

In [17]:
import re 
elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'
pattern = '(M[a-z]*) ([A-Za-z]*) ([0-9]*)'  #notare gli spazi bianchi tra i gruppi
mn = re.finditer(pattern, elenco)
#calcola tutte le sottostringhe che collimano con il pattern.
#ogni sottostringa è salvata in un match con un elemento per ogni gruppo

for m in mn: #m è il match contenente un singolo risultato
    print('nome: ' + m.group(1) + ' ' \
          'cognome: ' + m.group(2) + ' ' \
          'telefono: ' + m.group(3))

nome: Mario cognome: Bianchi telefono: 82342342
nome: Marco cognome: Verdi telefono: 342342342


In [22]:
#per scandire tutto l'elenco usiamo il seguente pattern
pattern = '([A-Z][a-z]*) ([A-Za-z]*) ([0-9]*)' 
mn = re.finditer(pattern, elenco)
for m in mn: #m è il match contenente un singolo risultato
    print('nome: ' + m.group(1) + ' ' \
          'cognome: ' + m.group(2) + ' ' \
          'telefono: ' + m.group(3))

nome: Luca cognome: Rossi telefono: 2345234
nome: Mario cognome: Bianchi telefono: 82342342
nome: Marco cognome: Verdi telefono: 342342342


### Riassumendo
Le notazioni e le funzioni che abbiamo visto sono:

- `[caratteri]` per definire un insieme di caratteri. La collimazione avviene con un solo carattere dell'insieme
- `[^caratteri]` lo stesso ma al contrario
- `*` sequenza  0 o  più occorrenze
- `?` sequenza 0 o 1 occorrenza
- `+` sequenza 1 o più occorrenze
- `(gruppo)` isola parti della stringa che collima
- `re.search` per trovare la prima stringa
- `re.finditer` per trovare tutte le stringhe che collimano e restituisce un oggetto iterabile
- `m.group()` per stampare tutta o parte della stringa trovata

In realtà Python permette di usare molti più caratteri speciali per definire le espressioni regolari, alcuni li menzioneremo più avanti, una lista più esaustiva la trovate in questo [Notebook](CaratteriSpeciali.ipynb).

### Tornando sulle differenze tra teoria delle espressioni regolari e applicazione in Python
Nella teoria, le espressioni regolari vengono usate per definire insiemi di stringhe. Ad esempio, l'espressione regolare **a\[a-z\]\*c** denota l'insieme contenente ogni stringa che inizia per **a**, seguita da un numero qualunque (anche 0) di altre lettere e poi terminata da una **c**. La definizione è precisa e non ambigua, ma quando questa espressione regolare viene usata per cercare una stringa di questo insieme all'interno di un testo, ci sono varie possibilità. Vediamo un esempio e come si comporta il modulo *re* di Python e le sue funzioni.

Assumiamo che il nostro pattern sia quello descritto sopra e vediamo il comportamento in 2 casi:

In [23]:
pattern = 'a[a-z]*ll[a-z]*to'
s1 = 'appallottolato'
s2 = 'appallottolato o imballatto'

##### Applichiamo i metodi search e finditer
Applicando i 2 metodi nelle 2 situazioni otteniamo:

In [24]:
print('ricerca nel testo con pattern',pattern,s1)
m = re.search(pattern,s1)
if m:
    print('search  ',m.group())

mn = re.finditer(pattern,s1)
for m in mn:
    print('finditer',m.group())
    
print() #stampa riga di separazione

print('ricerca nel testo con pattern',pattern,s2)
m = re.search(pattern,s2)
if m:
    print('search  ',m.group())

mn = re.finditer(pattern,s2)
for m in mn:
    print('finditer', m.group())

ricerca nel testo con pattern a[a-z]*ll[a-z]*to appallottolato
search   appallottolato
finditer appallottolato

ricerca nel testo con pattern a[a-z]*ll[a-z]*to appallottolato o imballatto
search   appallottolato
finditer appallottolato
finditer allatto


**Osservazioni sull'esempio sopra:**
Come si vede, nel caso di `s1` sia la search che la finditer hanno trovato una sola stringa ed è la stringa **più lunga** dell'insieme delle stringhe che collimano con l'espressione regolare, anche se c'erano altre stringhe in `s1` che collimavano con il pattern, come ad esempio 'allotto', 'appallotto' o 'allottolato'. Nel caso di `s2` la finditer trova 2 soluzioni distinte. Nel prossimo paragrafo speghiamo più precisamente il comportamento della `search` e della `finditer`.

## Metodi fondamentali per la ricerca
Di seguito forniamo l'elenco dei pricipali metodi per la ricerca, spiegandone il comportamento.

* `match(pattern, string, flags=0)`	Determina se la RE (regular expression) *pattern* corrisponde (match) **all'inizio** della stringa *string* e restituisce un oggetto di tipo match, che, intuitivamente, è un oggetto che rappresenta la corrispondenza trovata. Restituisce `None` se nessuna corrispondenza è stata trovata. 
* `search(pattern, string, flags=0)`	Analizza la stringa *string* cercando la **prima posizione** in cui la RE *pattern* produce una corrispondenza e restituisce l'oggetto di tipo match corrispondente. Restituisce `None` se nessuna posizione nella stringa corrisponde con la RE. **N.B.**  Cerca la **prima (più a sinistra)** sottostringa di testo che collima con l'espressione regolare e, se ci sono più stringhe che collimano ed iniziano nella stessa posizione, prende **la più lunga**.
* `findall(pattern, string, flags=0)`	Restituisce tutte le corrispondenze **non sovrapposte** della RE *pattern* nella stringa *string*, sotto forma di una lista di stringhe. La stringa *string* viene scansionata da sinistra a destra e le stringhe che collimano vengono restituite in una lista, nell'ordine in cui sono trovate. Se uno o più gruppi sono presenti nel *pattern*, restituisce una lista di tuple di stringhe (la cui componente i-esima corrisponde al gruppo i). **N.B.** Si comporta come la `search` per cercare ogni soluzione e quando ne trova una riparte a cercarne altre **dalla posizione successiva a quelle occupate dalla precedente soluzione**, cioè cerca tutte soluzioni che **non abbiano alcuna sovrapposizione**.
* `finditer(pattern, string, flags=0)`	Trova tutte le corrispondenze **non sovrapposte** della RE *pattern* nella stringa *string*, e le restituisce in un iteratore che produce un oggetto di tipo match alla volta. La stringa *string* viene scansionata da sinistra a destra e gli oggetti di tipo match vengono restituiti nell'ordine trovato. **N.B.** Per quanto riguarda la ricerca, si comporta come la `findall`.

Le principali opzioni disponibili (*flags*) sono: 
* `re.DOTALL` Permette al carattere speciale **'.'** (vedi prossima cella) di collimare con ogni carattere, includendo il fine riga ('\n'), altrimenti non considerato;
* `re.IGNORECASE`	Cerca la corrispondenza dei caratteri senza fare distinzione tra maiuscolo e minuscolo
* `re.MULTILINE`  Tratta ogni riga come se fosse l'inizio della stringa, quindi **'^'** collima con l'inizio di ogni riga e **'\$'** collima con la fine di ogni riga.

Per specificare più di una opzione, si utilizza l'operatore | per connetterle. Per esempio

`re.search(pattern,string,flags=re.IGNORECASE|re.MULTILINE)`

`flags=` può essere omesso, ottenendo quindi

`re.search(pattern,string,re.IGNORECASE|re.MULTILINE)`

Ovviamente, se l'importazione della libreria `re` è avvenuta con il comando 

`from re import *`

è possibile anche scrivere `DOTALL` al posto di `re.DOTALL` (analogamente per le altre opzioni).

I metodi `match()` e `search()` restituiscono un oggetto di tipo `Match`, mentre la funzione `finditer()` restituisce un iteratore che genera un oggetto di tipo `Match` per volta. Gli oggetti di tipo `Match` hanno questi metodi fondamentali:

| Metodo |	Scopo |
|:---:|:---:|
|`group()` |	Restituisce la stringa che collima con RE |
|`start()` |	Restituisce la posizione iniziale della sottostringa che collima con RE |
| `end()` |	Restituisce la posizione successiva alla fine della sottostringa che collima con RE|
|`span()`	| Restituisce la tupla con le posizioni restituite da `start()` e `end()` |

## Caratteri denotanti gruppi e posizioni
Alcune delle sequenze speciali che iniziano con "\\" rappresentano degli insiemi predefiniti di caratteri che sono spesso utili, come l'insieme delle cifre, l'insieme delle lettere o l'insieme di tutto ciò che non è un carattere di spaziatura. Sono disponibili le seguenti sequenze predefinite:

* **\d** Corrisponde ad ogni cifra decimale; equivale alla classe \[0-9\].
* **\D** Corrisponde ad ogni carattere che non sia una cifra; è equivalente alla classe \[^0-9\].
* **\s** Corrisponde ad ogni carattere di spaziatura; è equivalente alla classe \[ \t\n\r\f\v\] (si ricorda che \t è un tab orizzontale, \n è una andata a capo, \r è un carriage return - riporta il cursore all'inizio della riga, \f è un form feed - avanza la stampa alla pagina successiva, \v è un tab verticale).
* **\S** Corrisponde ad ogni carattere che non sia un carattere di spaziatura; è equivalente alla classe \[^\t\n\r\f\v\].
* **\w** Corrisponde ad ogni carattere alfanumerico (più l'underscore '\_'); è equivalente alla classe \[a-zA-Z0-9\_\].
* **\W** Corrisponde ad ogni carattere che non sia alfanumerico e non sia l'underscore; è equivalente alla classe \[^a-zA-Z0-9\_\].
* **.** Corrisponde ad ogni carattere diverso da '\n'. Se usato con il flag re.DOTALL corrisponde ad ogni carattere.
* **\b** Corrisponde con la stringa vuota, ma solo se questa si trova all'inizio o alla fine di una parola (cioè di una sequenza di caratteri \w). In altri termini, \b indica il "confine" (boundary) di una parola, che tecnicamente è quello che si trova fra un carattere \w ed un carattere \W (o vice-versa) o fra \w e l'inizio/fine della stringa. (si noti che in questo caso nella stringa che rappresenta l'espressione regolare dobbiamo scrivere `\\b`, altrimenti la sequenza speciale sarebbe interpretata come il carattere backspace; in alternativa dobbiamo usare usare una stringa raw).
* **^** Corrisponde con la stringa vuota, ma solo se questa si trova all'inizio della stringa. Denota quindi l'inizio della stringa. Se usato con il flag re.MULTILINE, corrisponde all'inizio di una riga.
* **$** Corrisponde con la stringa vuota, ma solo se questa si trova alla fine della stringa. Denota quindi la fine della stringa. Se usato con il flag re.MULTILINE, corrisponde alla fine di una riga.
* **"*?", "+?", "??"** I qualificatori "\*", "+" e "?" sono ''ingordi'', cioè corrispondono a tutto il testo possibile. A volte questo comportamento non è quello desiderato; se l'espressione regolare \<.\*\> viene fatta corrispondere alla stringa ```<H1>title</H1>```, corrisponderà all'intera stringa (longest match) e non a ```<H1>``` e ```</H1>```. Aggiungere il carattere "?" dopo il qualificatore rende la ricerca delle corrispondenze non-ingorda o minimale; infatti verrà fatto corrispondere il minor numero possibile di caratteri. Usando \<.\*?\> nella precedente espressione vi saranno due corrispondenze, una con ```<H1>``` ed una con ```</H1>```.

**Nota:** Da qui in avanti, con parola intendiamo PER DEFINIZIONE una sequenza di caratteri \w.

## Ricerca di stringhe con ripetizioni

* **\\numero** Corrisponde al contenuto del **gruppo** con lo stesso numero. I gruppi vengono numerati a partire da 1. Per esempio, '(.+)_\\\1' è una espressione regolare che corrisponde, ad esempio, a 'the_the' o a '55_55', ma non a 'the_sun' o a 'thethe'. Notate ancora una volta che il carattere '\\' va ripetuto 2 volte, per distinguere dal caso in cui è usato per specificare una sequenza di escape. Se volete evitare la ripetizione potete usare la stringa *raw* r'(.+) \1'. In altri termini, nell'espressione precedente, (.+) corrisponde ad una sequenza di un numero arbitrario ma maggiore di 1 di caratteri qualsiasi; una volta individuato questo gruppo, \1 corrisponde ad una sua replica (che nell'espressione regolare dell'esempio deve essere separata dalla precedente da un underscore).

In [77]:
import re 

elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342 - Gemma Neri 82342342'
#stampa il nome e il cognome di una coppia di persone che hanno lo stesso numero (se c'è)
pattern = r'([A-Z][a-z]*) ([A-Z][a-z]*) ([0-9]*) .* ([A-Z][a-z]*) ([A-Z][a-z]*) \3 ?' # nota la r davanti!
m=re.search(pattern,elenco)
if m:
    print('Nome:',m.group(1),' ','Cognome:',m.group(2))
    print('Nome:',m.group(4),' ','Cognome:',m.group(5))
else:
    print('Non trovata!')

Nome: Mario   Cognome: Bianchi
Nome: Gemma   Cognome: Neri


In [ ]:
import re 

elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342 - Gemma Neri 82342342'
#stampa il nome e il cognome di una coppia di persone che hanno lo stesso numero (se c'è)
pattern = r'([A-Z][a-z]*) ([A-Z][a-z]*) ([0-9]*) .* ([A-Z][a-z]*) ([A-Z][a-z]*) \3 ?' # nota la r davanti!
m=re.search(pattern,elenco)
if m:
    print('Nome:',m.group(1),' ','Cognome:',m.group(2))
    print('Nome:',m.group(4),' ','Cognome:',m.group(5))
else:
    print('Non trovata!')

## Esempi di esercizi semplici

In [43]:
import re
testo=open("I_Malavoglia.txt",encoding="UTF-8").read()
# Cerchiamo nel file "I_Malavoglia.txt" tutte le parole
l=re.findall(r'\b\w+\b',testo,re.IGNORECASE)
print(l[:100])

['I', 'Malavoglia', 'Giovanni', 'Verga', 'Esportato', 'da', 'Wikisource', 'il', '12', '11', '2018', 'Segnala', 'eventuali', 'errori', 'altre', 'amenita', 'su', 'it', 'wikisource', 'org', 'wiki', 'Segnala_errori', 'I', 'VINTI', 'I', 'MALAVOGLIA', 'ROMANZO', 'di', 'Giovanni', 'Verga', 'MILANO', 'fratelli', 'treves', 'editori', '1907', 'Sesto', 'Migliaio', 'Indice', 'Prefazione', 'Capitolo', 'I', 'Capitolo', 'II', 'Capitolo', 'III', 'Capitolo', 'IV', 'Capitolo', 'V', 'Capitolo', 'VI', 'Capitolo', 'VII', 'Capitolo', 'VIII', 'Capitolo', 'IX', 'Capitolo', 'X', 'Capitolo', 'XI', 'Capitolo', 'XII', 'Capitolo', 'XIII', 'Capitolo', 'XIV', 'Capitolo', 'XV', 'Questo', 'racconto', 'è', 'lo', 'studio', 'sincero', 'e', 'spassionato', 'del', 'come', 'probabilmente', 'devono', 'nascere', 'e', 'svilupparsi', 'nelle', 'più', 'umili', 'condizioni', 'le', 'prime', 'irrequietudini', 'pel', 'benessere', 'e', 'quale', 'perturbazione', 'debba', 'arrecare', 'in', 'una']


In [30]:
import re
testo=open("I_Malavoglia.txt",encoding="UTF-8").read()
# Cerchiamo nel file "I_Malavoglia.txt" tutte le parole che iniziano e finiscono con a, aa compresa
l=re.findall(r'\ba\w*a\b',testo,re.IGNORECASE)
print(l[:100])

['amenita', 'allora', 'ancora', 'ancora', 'aristocratica', 'allarga', 'artistica', 'alla', 'accompagna', 'avuta', 'alla', 'artista', 'altra', 'acqua', 'ammarrata', 'alla', 'alla', 'ammarrata', 'alla', 'aveva', 'Agata', 'ancora', 'Alla', 'aveva', 'allora', 'avea', 'alla', 'Allora', 'Allora', 'altra', 'ancora', 'aveva', 'andava', 'alla', 'alla', 'alla', 'aveva', 'andava', 'aveva', 'Alla', 'ancora', 'amara', 'acqua', 'andava', 'alla', 'alla', 'abbia', 'aveva', 'ancora', 'aveva', 'andava', 'apposta', 'allora', 'aveva', 'aveva', 'andava', 'aveva', 'aveva', 'aveva', 'andava', 'appuntava', 'andava', 'annata', 'anima', 'alla', 'ancora', 'andava', 'aveva', 'aveva', 'acqua', 'ammarrata', 'anima', 'avveniva', 'aveva', 'aperta', 'andava', 'aveva', 'apriva', 'affaccendava', 'aveva', 'anima', 'alla', 'avemaria', 'alla', 'appesa', 'antenna', 'anitroccola', 'aspetta', 'andarla', 'altra', 'Allora', 'alla', 'aveva', 'allora', 'acqua', 'aspetta', 'aveva', 'aspetta', 'acqua', 'alla']


In [36]:
import re

# Cerchiamo nel file "I_Malavoglia.txt" la più lunga stringa INIZIALE 
# che inizia per una vocale e finisce per la STESSA vocale, 
# ignorando maiuscole e minuscole
testo=open("I_Malavoglia.txt",encoding="UTF-8").read()
regex1 = r'^([aeiou]).*\1'
ris1 = re.search(regex1,testo,re.IGNORECASE)
print(ris1.span(),ris1.group())
print()


#attenzione! la stessa cosa fatta con la findall:
l = re.findall(regex1,testo,re.IGNORECASE)
print('findall',l[:10])

#nella lista l vengono riportate SOLO le parti di stringa corrispondenti a gruppi ()

l = re.findall(r'^([aeiou]).*\1',testo,re.IGNORECASE)
print('findall2',l[:10])
print()

# Cerchiamo nel file "I_Malavoglia.txt" la prima riga che INIZIA 
# con una vocale e finisce per la STESSA vocale seguita 
# dal punto e dal carattere di riga nuova '\n', 
# ignorando maiuscole e minuscole
regex2 = r'^([aeiou]).*?\1\.\n'
ris2 = re.search(regex2,testo,re.IGNORECASE | re.MULTILINE)
print(ris2.span(),ris2.group())

(0, 11) I Malavogli

findall ['I']
findall2 ['I']

(2965, 4379) Il cammino fatale, incessante, spesso faticoso e febbrile che segue l’umanità per raggiungere la conquista del progresso, è grandioso nel suo risultato, visto nell’insieme, da lontano. Nella luce gloriosa che l’accompagna dileguansi le irrequietudini, le avidità, l’egoismo, tutte le passioni, tutti i vizi che si trasformano in virtù, tutte le debolezze che aiutano l’immane lavoro, tutte le contraddizioni, dal cui attrito sviluppasi la luce della verità. Il risultato umanitario copre quanto c’è di meschino negli interessi particolari che lo producono; li giustifica quasi come mezzi necessari a stimolare l’attività dell’individuo cooperante inconscio a beneficio di tutti. Ogni movente di cotesto lavorìo universale, dalla ricerca del benessere materiale, alle più elevate ambizioni, è legittimato dal solo fatto della sua opportunità a raggiungere lo scopo del movimento incessante; e quando si conosce dove vada questa immensa c

Si noti che nell'ultimo esempio il primo `.` è da intendersi come il simbolo speciale che nelle espressioni regolari corrisponde a qualunque carattere, mentre il secondo `.` corrisponde effettivamente ad un punto nella stringa. Per distinguere i due casi (ed intendere il secondo punto come carattere e non come simbolo speciale) abbiamo utilizzato il `\` prima del secondo punto (avremmo fatto lo stesso per cercare nel stringa un asterisco (`*`) o un qualunque altro carattere che è utilizzato come simbolo speciale nell'espressione regolare).

In [37]:
# Contiamo nel file "I_Malavoglia.txt" quante righe INIZIANO per una
# vocale e finiscono per la STESSA vocale seguita dal punto e dal 
# carattere di riga nuova '\n', ignorando maiuscole e minuscole
regex2 = r'^([aeiou]).*\1\.\n'
ris3 = re.findall(regex2,testo,re.IGNORECASE | re.MULTILINE)
print(len(ris3))
for i in range(len(ris3)):
    print(i,ris3[i])


31
0 I
1 E
2 A
3 I
4 A
5 I
6 O
7 I
8 O
9 E
10 I
11 I
12 A
13 A
14 E
15 A
16 E
17 A
18 A
19 E
20 E
21 A
22 E
23 I
24 A
25 I
26 A
27 I
28 A
29 I
30 E


In [38]:
# Cerchiamo nel file "I_Malavoglia.txt" tutte le righe (paragrafi) che INIZIANO 
# per una vocale e finiscono per la STESSA vocale seguita dal punto e
# dal carattere di riga nuova '\n', ignorando maiuscole e minuscole
regex2 = r'^([aeiou]).*\1\.\n'
ris4 = re.finditer(regex2,testo,re.IGNORECASE | re.MULTILINE)
for m in ris4:
    print(m.span(),m.group())
    

(2965, 4379) Il cammino fatale, incessante, spesso faticoso e febbrile che segue l’umanità per raggiungere la conquista del progresso, è grandioso nel suo risultato, visto nell’insieme, da lontano. Nella luce gloriosa che l’accompagna dileguansi le irrequietudini, le avidità, l’egoismo, tutte le passioni, tutti i vizi che si trasformano in virtù, tutte le debolezze che aiutano l’immane lavoro, tutte le contraddizioni, dal cui attrito sviluppasi la luce della verità. Il risultato umanitario copre quanto c’è di meschino negli interessi particolari che lo producono; li giustifica quasi come mezzi necessari a stimolare l’attività dell’individuo cooperante inconscio a beneficio di tutti. Ogni movente di cotesto lavorìo universale, dalla ricerca del benessere materiale, alle più elevate ambizioni, è legittimato dal solo fatto della sua opportunità a raggiungere lo scopo del movimento incessante; e quando si conosce dove vada questa immensa corrente dell’attività umana, non si domanda al cert

## Funzione per la sostituzione
Ancora più utili possono essere le espressioni regolari quando si vogliono fare complesse sostituzioni. La funzione fondamentale per effettuare sostituzioni usando le espressioni regolari è:
* `sub(pattern, repl, string, count=0, flags=0)` Restituisce una nuova stringa dove tutte le occorrenze **non sovrapposte** dell'espressione regolare (*pattern*) nella stringa (*string*) sono sostituite dal rimpiazzo (*repl*), anche esso specificato con un'espressione regolare. Il parametro *count* serve per limitare (eventualmente) il numero di sostituzioni da effettuare.

Vediamo come esempio la modifica delle istruzioni print dalla versione 2 alla 3 di Python, vedi il file [prova.py](prova.py):

In [39]:
import re

# Modifica le istruzioni print di Python2 nella versione corretta per Python3
filein = open('prova.py',encoding='UTF-8')
fileout = open('nuovoprova.py','w',encoding='UTF-8')
pattern = 'print (.*)\n'
repl = r'print(\1)\n'
#for riga in filein:
#    if 'print' in riga:
#        nuova = re.sub(pattern,repl,riga)
#        print(nuova)
#        fileout.write(nuova)
#    else:
#        fileout.write(riga)
for riga in filein:
    nuova = re.sub(pattern,repl,riga)
    if nuova!=riga:
        print(nuova)
    fileout.write(nuova)
filein.close()
fileout.close()

	print(i)

	print(j, a[j])



#### Discussione
il programma di sopra funziona correttamente nella maggior parte dei casi, ma fallisce sull'ultima riga del file [prova.py](prova.py), in quanto l'ultima riga NON è terminata da un '\n'. Per rendere più completa la nostra soluzione dobbiamo prevedere l'opzionalità del '\n' finale, ma allo stesso tempo stare attenti ad ottenere un risultato in cui non viene modificata la struttura del file originario. Per fare questo introduciamo un secondo gruppo che include il solo '\n' finale (se presente) oppure la stringa vuota. Questo gruppo viene inserito alla fine della relativa istruzione di print.

In [40]:
# Modifica le istruzioni print di Python2 nella versione corretta per Python3
filein = open('prova.py',encoding='UTF-8')
fileout = open('nuovoprova2.py','w',encoding='UTF-8')
pattern = 'print (.*)(\n?)'
repl = r'print(\1)\2'
for riga in filein:
    if 'print' in riga:
        nuova = re.sub(pattern,repl,riga)
        print(nuova)
        fileout.write(nuova)
    else:
        fileout.write(riga)
filein.close()
fileout.close()

	print(i)

	print(j, a[j])

print('Basta!')


## Esempi di esercizi più complessi

### Esercizio 1
Scrivere una funzione che riceve il nome di un file di testo e trova la prima sequenza di due parole consecutive che iniziano per lo stesso carattere. 

In [6]:
def trovaParole(file):
    import re
    f = open(file,encoding='UTF-8')
    testo = f.read()
    f.close()
    # Il pattern deve essere fatto così:
    # inizio parola "\b" seguito da un carattere (alfanumerico o underscore) da memorizzare "(\w)",
    # seguito da altri caratteri "\w*" poi fine parola "\b", spazi o caratteri
    # non alfanumerici (o underscore) "\W*", inizio parola "\b" seguito dal carattere
    # memorizzato "(\1)", altri caratteri "\w*" e poi fine parola "\b"
    pattern = r'\b(\w)\w*\b\W*\b\1\w*\b'
    ris = re.search(pattern,testo,re.IGNORECASE )
    return ris

print(trovaParole('I_Malavoglia.txt').group())



12/11


### Esercizio 2
Simile a prima ma trova **tutte** le sequenze **non sovrapposte** di **tre** parole consecutive che iniziano tutte per lo stesso carattere.

In [119]:
def trovaParole2(file):
    import re
    f = open(file,encoding='UTF-8')
    testo = f.read()
    f.close()
    pattern = r'\b(\w)\w*\b\W*\b\1\w*\b\W*\b\1\w*\b'
    #pattern = r'\b(\w)(\w*\b\W*\b\1){2}\w*\b' # attenzione: l'uso delle parentesi introduce un gruppo!
    ris = re.finditer(pattern,testo,re.IGNORECASE )
    return ris

print(trovaParole2('I_Malavoglia.txt'))
for m in trovaParole2('I_Malavoglia.txt'):
    print(m.group())

certo come ci
la legge, lui
Cristoforo che c
si sarebbe soffiato
dire «donna di
di Dio di
segretario si smascellava
con coscienza, come
ancora ad Aci
stivalini stretti sul
esso, ed evitava
andare ad aspettarli
modo, mogi mogi
loro. La Longa
stecconato, senza saper
la Longa, le
colonna contro cui
della disciplina, dei
leggere la lettera
sentisse sullo stomaco
appesa all’antenna
sedersi sugli scalini
Sulla strada si
lettere leggeva la
come compare Cipolla
Ah! ah! ah
Cipolla confermò che
certi cannelli che
carne che ci
morto mio marito
Alfio andava a
letto.

La Longa
si sentiva sullo
sasso, si stringeva
Giammaria gli gridava
della disputa don
sputava sentenze: — Sentite
ad ascoltare a
così cieco com
sobbalzando sui sassi
Spatu si sgolava
sulla sabbia, si
andata ad aspettare
colle colonne, che
che contava come
ad andare attorno
cose che costavano
cervello corto», chè
dei Dolori di
piove più perchè
ah! ah! ah
all’anno. Allora
altro.

— Allora, — aggiunse
Crocifisso che ci
deve dire di
compa

### Esercizio 3
Scrivere una funzione che riceve il nome di un file di testo e un carattere c e trova la prima sequenza di tre parole consecutive che iniziano tutte per c.

In [9]:
def trovaParole3(file,c):
    import re
    f = open(file,encoding='UTF-8')
    testo = f.read()
    f.close()
    pattern = r'\b'+c+r'\w*\b\W*\b'+c+r'\w*\b\W*\b'+c+r'\w*\b'
    ris = re.search(pattern,testo,re.IGNORECASE)
    return ris

print(trovaParole3('I_Malavoglia.txt','s').group())

si sarebbe soffiato


### Esercizio 4
Scrivere una funzione che riceve il nome di un file di testo, un carattere c ed un numero n (maggiore di 1) e trova la prima sequenza di n parole consecutive che iniziano tutte per c.

In [101]:
def trovaParole4(file,c,n):
    import re
    f = open(file,encoding='UTF-8')
    testo = f.read()
    f.close()
    pattern = r'\b'+c+(r'\w*\b\W*\b'+c)*(n-1)+r'\w*\b'
    ris = re.search(pattern,testo,re.IGNORECASE)
    return ris

print(trovaParole4('I_Malavoglia.txt','e',3).group())

esso, ed evitava


### Esercizio 5
Trovate tutte le righe di un file che, dopo averle pulite con lo `strip()`, iniziano e finiscono per lo stesso carattere (ignorando la differenza fra maiuscole e minuscole), seguito eventualmente dal punto, ed inseritele in una lista.

In [102]:
def listaRighe(file):
    filein = open(file,encoding='UTF-8')
    pattern = r'^(\w).*\1\.?$'
    lista = []
    for riga in filein:
        rigaPulita = riga.strip()
        if re.search(pattern,rigaPulita,re.IGNORECASE):
            lista.append(rigaPulita)
    filein.close()
    return lista

ris = listaRighe("I_Malavoglia.txt")
print(len(ris),ris[:10])


32 ['I VINTI', 'Il cammino fatale, incessante, spesso faticoso e febbrile che segue l’umanità per raggiungere la conquista del progresso, è grandioso nel suo risultato, visto nell’insieme, da lontano. Nella luce gloriosa che l’accompagna dileguansi le irrequietudini, le avidità, l’egoismo, tutte le passioni, tutti i vizi che si trasformano in virtù, tutte le debolezze che aiutano l’immane lavoro, tutte le contraddizioni, dal cui attrito sviluppasi la luce della verità. Il risultato umanitario copre quanto c’è di meschino negli interessi particolari che lo producono; li giustifica quasi come mezzi necessari a stimolare l’attività dell’individuo cooperante inconscio a beneficio di tutti. Ogni movente di cotesto lavorìo universale, dalla ricerca del benessere materiale, alle più elevate ambizioni, è legittimato dal solo fatto della sua opportunità a raggiungere lo scopo del movimento incessante; e quando si conosce dove vada questa immensa corrente dell’attività umana, non si domanda al c

### Esercizio 6
Contare quante volte in un file compare una sequenza di 3 parole consecutive con la seguente proprietà: la lettera finale della prima parola è uguale a quella iniziale della seconda e quella finale della seconda è uguale a quella iniziale della terza.
**Nota:** Si assuma che la seconda parola di ciascuna sequenza contenga almeno due lettere

In [71]:
from tester import tester_fun
import re

def conta(file):
    f=open(file,"r",encoding="UTF-8")
    testo=f.read()
    f.close()
    regex = r'\b\w*(\w)\b\W*\b\1\w*(\w)\b\W*\b\2\w*\b'
    ris = re.findall(regex,testo,re.IGNORECASE)
    return len(ris)

counter_test_positivi = 0

counter_test_positivi += tester_fun(conta,['file1.txt'],2)
counter_test_positivi += tester_fun(conta,['file2.txt'],3)
counter_test_positivi += tester_fun(conta,['file3.txt'],4)

print('La funzione',conta.__name__,'ha superato',counter_test_positivi,'test')

Test funzione: conta 

Input funzione: 'file1.txt' 

Output atteso:
2

----- print interne funzione -----

----------------------------------

Output ottenuto:
2

Risultato Test: POSITIVO

******************************

Test funzione: conta 

Input funzione: 'file2.txt' 

Output atteso:
3

----- print interne funzione -----

----------------------------------

Output ottenuto:
3

Risultato Test: POSITIVO

******************************

Test funzione: conta 

Input funzione: 'file3.txt' 

Output atteso:
4

----- print interne funzione -----

----------------------------------

Output ottenuto:
4

Risultato Test: POSITIVO

******************************

La funzione conta ha superato 3 test


### Esercizio 7
Calcolare quante volte in un file compare una sequenza di 2 parole consecutive con la seguente proprietà: Almeno 2 lettere della prima parola sono presenti anche nella seconda e nello stesso ordine, cioè se la prima lettera viene prima della seconda nella prima parola, deve venire prima anche nella seconda.

**Nota:** non si considerino sequenze sovrapposte (ad esempio, nella stringa 'asta sito toro' c'è comunque una sola sequenza, perchè 'sito' deve essere considerato parte di una sola sequenza; in questo esempio è corretto quindi contare solo la prima sequenza asta sito')

In [138]:
from tester import tester_fun

import re

def conta2(file):
    f=open(file,"r",encoding="UTF-8")
    testo=f.read()
    f.close()
    regex = r'\b\w*(\w)\w*(\w)\w*\b\W*\b\w*\1\w*\2\w*\b'
    ris = re.findall(regex,testo,re.IGNORECASE)
    print(ris.group())
    return len(ris)

counter_test_positivi = 0

counter_test_positivi += tester_fun(conta2,['file1.txt'],1)
counter_test_positivi += tester_fun(conta2,['file2.txt'],1)
counter_test_positivi += tester_fun(conta2,['file3.txt'],2)

print('La funzione',conta2.__name__,'ha superato',counter_test_positivi,'test') 


Test funzione: conta2 

Input funzione: 'file1.txt' 

Output atteso:
1

----- print interne funzione -----

----------------------------------

La funzione ha lanciato una eccezione durante il test

Traceback (most recent call last):
  File "c:\Users\campe\Desktop\Vittorio UNI\Programazione\15EspressioniRegolari-20231201T175217Z-001\15EspressioniRegolari\tester.py", line 97, in helper_tester_function
    returned_value = function(*input_data)
                     ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\campe\AppData\Local\Temp\ipykernel_12852\1624543690.py", line 11, in conta2
    print(ris.group())
          ^^^^^^^^^
AttributeError: 'list' object has no attribute 'group'

Risultato Test: NEGATIVO

******************************

Test funzione: conta2 

Input funzione: 'file2.txt' 

Output atteso:
1

----- print interne funzione -----

----------------------------------

La funzione ha lanciato una eccezione durante il test

Traceback (most recent call last):
  File "c:\Users\campe\Des

### Esercizio 8
Vogliamo stampare tutte le date presenti in un file, assumiamo che le date siano nel formato 21/11/2013. Useremo come test il file 'testoDateCF.txt'.

In [44]:
file = open('testoDateCF.txt', encoding = 'UTF-8')
testo = file.read()

In [45]:
# Prima versione con formato rigido
pattern1 = '[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]'
i = re.finditer(pattern1,testo)
for m in i:
    print(m.group())

21/11/2016
12/11/2016
11/12/2011


In [46]:
# Seconda versione giorno e mese possono avere anche una sola cifra
pattern2 = '[0-9]?[0-9]/[0-9]?[0-9]/[0-9][0-9][0-9][0-9]'
i = re.finditer(pattern2,testo)
for m in i:
    print(m.group())

21/11/2016
12/11/2016
2/11/2016
21/2/2014
7/2/2014
11/12/2011


In [47]:
# Terza versione giorno e mese possono avere anche una sola cifra e 
# l'anno può avere 2 o 4 cifre
pattern2 = '[0-9]?[0-9]/[0-9]?[0-9]/([0-9][0-9])?[0-9][0-9]'
i = re.finditer(pattern2,testo)
for m in i:
    print(m.group())
    

21/11/2016
12/11/2016
2/11/2016
21/2/2014
7/2/2014
11/12/2011
1/1/11
1/11/91


### Esercizio 9 
Scrivere una funzione che riceve in ingresso il nome di un file e restituisce due liste, ordinate alfabeticamente, di tutti i codici fiscali **corretti** presenti nel file, la prima contenente i codici fiscali degli uomini e la seconda quelli delle donne.

Il formato del codice fiscale è:
1. tre maiuscole
2. spazio (opzionale)
3. tre maiuscole
4. spazio (opzionale)
5. due cifre, una maiuscola, due cifre
6. spazio (opzionale)
7. maiuscola, tre cifre, maiuscola

Il gruppo 5 denota la data di nascita, le prime 2 cifre reppresentano l'anno, la lettera denota il mese e le ultime 2 cifre denotano il giorno di nascita. Per le donne, il giorno di nascita viene aumentato di 40. Per verificare la correttezza di questo gruppo dovete solo verificare che il giorno di nascita sia un valore compreso tra 1 e 31, per gli uomini, oppure tra 40 e 71, per le donne, (quindi adottate l'assunzione semplificatrice che tutti i mesi siano di 31 giorni) e che la lettera relativa al mese sia una tra le lettere 'ABCDEHLMPRST' che rappresentano i 12 mesi dell'anno.

In [87]:
# La soluzione proposta prima estrae tutte le stringhe che hanno
# la struttura del codice fiscale, verificando poi se siano corretti
# e decidendo se si riferiscono a uomini o donne.
import re

def trovaCF(file):
    #definiamo il pattern seguendo le regole viste sopra
    #Notate le parentesi intorno alle cifre del giorno e alla cifra
    #del mese, servono per estrarre questi campi e poterli analizzare
    #per poter verificare correttezza e sesso.
    pattern='[A-Z]{3} ?[A-Z]{3} ?[0-9]{2}([A-Z])([0-9]{2}) ?[A-Z][0-9]{3}[A-Z]'
    s = open(file).read()
    i = re.finditer(pattern, s)
    l_u = []
    l_d = []
    mesi = 'ABCDEHLMPRST'
    for m in i:
        cf = m.group(0)
        giorno = int(m.group(2))
        mese = m.group(1)
        if (1 <= giorno <= 31) or (41 <= giorno <= 71) and mese in mesi:
            if giorno <= 31:
                l_u.append(cf)
            else:
                l_d.append(cf)
    l_u.sort()
    l_d.sort()
    return l_u,l_d

trovaCF('testoDateCF.txt')

(['CCCMRC17A11H501W', 'CCCMRC72Z11H501W', 'CCDMRC55P13H501W'],
 ['CCCMRC00M57H501W', 'CCCMRC96E41H501W', 'CCCSRC08S71H501W'])

#### Discussione
Il programma di sopra risolve il problema, ma non è completamente preciso. Per essere precisi, bisognerebbe verificare che il numero sia coerente con il mese (Aprile, Giugno, Settembre e Novembre hanno 30 giorni e Febbraio ne ha 28 o 29). Nell'Esercizio 4 incluso nella lista degli esercizi finali vi viene chiesto di rendere il controllo più preciso sulle date.

### Esercizio 10
Dobbiamo scrivere una funzione che riceve una stringa contenente delle date nel formato dd/mm/yyyy (ammettendo anche che il giorno e mese possano essere scritti con una sola cifra e l'anno con sole due cifre) e le scrive nel formato americano con solo 2 cifre per gli anni (mm/dd/yy)

In [90]:
def converti(s):
    pattern = '(\d?\d)/(\d?\d)/(\d\d)?(\d\d)'
    repl = r'\2/\1/\4'
    return re.sub(pattern,repl,s)

print(converti('01/02/1972 1/3/89 11/12/2011'))

02/01/72 3/1/89 12/11/11


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\campe\AppData\Local\Temp\ipykernel_12852\3403577371.py:2: SyntaxWarning: invalid escape sequence '\d'
  pattern = '(\d?\d)/(\d?\d)/(\d\d)?(\d\d)'


### Esercizio 11
Vogliamo convertire tutte le date ed ore presenti in un file nel formato americano con mesi per esteso (in lettere), gioni (in cifre) con i suffissi ed anno a 4 cifre (assumere che tutte le date si riferiscano al secolo XX e mettere quindi all'inizio 19, se necessario). Ad esempio, la data **11/07/89** deve diventare **July 11th 1989**. Per le ore, si accettano sia nel formato **14:00** che **13.20** e vengono convertite nel formato americano (formato 12h + am/pm) con i **:** come separatore. Vediamo prima la funzione ausiliaria che converte le ore:

In [ ]:
# Converte le ore in formato 24h con i **:** come separatore, nel formato 12h 
# + am/pm  (con lo stesso separatore). 
# Attenzione alle eccezioni per i casi della mezzanotte e mezzogiorno
# per cui le 00:20 diventano 12:20am e le 12:30 diventano 12:30pm
def oreAmericane(s):
    coppia = s.split(':')
    ore = int(coppia[0])
    minuti = coppia[1]
    if ore >= 13:
        oraAm = str(ore-12)
        ampm = 'pm'
    elif ore == 12:
        oraAm = '12'
        ampm = 'pm'
    elif ore == 0:
        oraAm = '12'
        ampm = 'am'
    else:
        oraAm = str(ore)
        ampm = 'am'
    if len(oraAm) == 1:
        oraAm = '0'+oraAm
    return oraAm+':'+minuti+ampm

print('21:10',oreAmericane('21:10'))
print('01:10',oreAmericane('01:10'))
print('00:10',oreAmericane('00:10'))
print('12:10',oreAmericane('12:10'))

#### Funzioni ausiliarie `month()` e `day()`
Scriviamo le funzioni ausiliarie che convertono il mese nel nome esteso inglese ed il giorno con il suffisso appropriato

In [48]:
#Converte il mese numerico nel nome in inglese
def month(s):
    n = int(s)
    months = ('January','February','March','April','May','June','July','August','September','October','November','December')
    return months[n-1]

#Estende il giorno, che è una stringa di 1 o di cifre, aggiungendo il suffisso corretto
def day(s):
    n = int(s[-1])
    if len(s) > 1:
        d = s[0] #Decina del giorno, se assente vale None
    else:
        d = None
    if n == 1 and d != '1': #Fa eccezione 11
        return s+'st'
    if n == 2 and d != '1': #Fa eccezione 12
        return s+'nd'
    if n== 3 and d != '1': #Fa eccezione 13
        return s+'rd'
    return s+'th'

print('04',month('04'))
print('6',month('6'))
print('21',day('21'))
print('11',day('11'))
print('23',day('23'))
print('5',day('5'))


04 April
6 June
21 21st
11 11th
23 23rd
5 5th


#### Funzione complessiva
Possiamo ora scrivere la funzione complessiva chiamata **inglesizza(s)**

In [ ]:
def inglesizza(filein,fileout):
    f1 = open(filein,encoding= 'UTF-8')
    f2 = open(fileout,'w',encoding= 'UTF-8')
    s = f1.read()
    patternDate = '(\d?\d)/(\d?\d)/(\d\d)?(\d\d)'
    patternOre = '(\d?\d)[:.](\d\d)'
    for match in re.finditer(patternDate,s):
        d = day(match.group(1))
        m = month(match.group(2))
        if match.group(3) == None:
            year = '19'+match.group(4)
        else:
            year = match.group(3)+match.group(4)
        nuova = m+' '+d+' '+year
        s = s.replace(match.group(),nuova)
    for match in re.finditer(patternOre,s):        
        nuova = oreAmericane(match.group(1)+':'+match.group(2))
        s = s.replace(match.group(),nuova)
    f2.write(s)
    f1.close()
    f2.close()

inglesizza('Esempio Date Ore.txt','risultato.txt')

## Esercizi
Completate questi esercizi prima di cominciare il prossimo argomento

#### Esercizio 1: 
Scrivere un programma che prende in ingresso il nome di un file e restituisce la lista (ordinata alfabeticamente e senza ripetizioni) delle parole che contengono al loro interno almeno 3 doppie, cioè 3 coppie di lettere uguali consecutive come ad esempio 'arrabbattarsi'. Non fate differenza tra maiuscole e minuscole.

In [83]:
from tester import tester_fun

import re

def trovaParole(file):
    fin=open(file,'r',encoding='UTF-8')
    testo=fin.read()
    parole_trovate=set()
    pattern=r'\b(\w*(\w)\2\w*(\w)\3\w*(\w)\4\w*)\b'
    m=re.findall(pattern,testo,re.IGNORECASE)
   
    for tupla in m:
        parola = tupla[0]
        parole_trovate.add(parola)
    fin.close()
    parole_trovate=list(parole_trovate)
    parole_trovate.sort()
    return parole_trovate
tester_fun(trovaParole,['I_Malavoglia_50.txt'],['scappellotto'])
tester_fun(trovaParole,['I_Malavoglia.txt'],['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio'])


Test funzione: trovaParole 

Input funzione: 'I_Malavoglia_50.txt' 

Output atteso:
['scappellotto']

----- print interne funzione -----

----------------------------------

Output ottenuto:
['scappellotto']

Risultato Test: POSITIVO

******************************

Test funzione: trovaParole 

Input funzione: 'I_Malavoglia.txt' 

Output atteso:
['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio']

----- print interne funzione -----

----------------------------------

Output ottenuto:
['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio']

Risultato Test: POSITIVO

******************************



1

#### Esercizio 2

Generalizzare l'esercizio 1 mettendo il numero di doppie come parametro, cioè scrivere una funzione che prende come parametro il nome di un file ed il numero (minimo) n di doppie che le parole devono contenere e restituisce la lista (ordinata alfabeticamente e senza ripetizioni) delle parole che contengono al loro interno almeno n doppie, cioè n coppie di lettere uguali consecutive. Non fate differenza tra maiuscole e minuscole.


In [93]:
from tester import tester_fun

import re

def trovaParole2(file,n):
    fin=open(file,'r',encoding='UTF-8')
    testo=fin.read()
    pat=r'\b(\w*(\w)\2\w*){' + str(n) + r'}\b'   
    fine=set()
    r=re.findall(pat,testo,re.IGNORECASE) 
    for t in r:
        s=t[0]
        fine.add(s)
    fine=list(fine)
    fine=sorted(fine)
    fin.close()
    return fine
tester_fun(trovaParole2,['I_Malavoglia_50.txt',3],['scappellotto'])
tester_fun(trovaParole2,['I_Malavoglia_50.txt',2],['addii', 'affrettati', 'ammarrata', 'apparecchiare', 'arricchirsi', 'arricchisci', 'arricchito', 'donnicciuole', 'fazzoletto', 'gruzzoletto', 'parommella', 'parrocchia', 'ricchezze', 'scappellotto', 'soddisfatti', 'soggetto', 'villaggio', 'vorrebbe', 'zuppidda'])
tester_fun(trovaParole2,['I_Malavoglia.txt',3],['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio'])


Test funzione: trovaParole2 

Input funzione: 'I_Malavoglia_50.txt', 3 

Output atteso:
['scappellotto']

----- print interne funzione -----

----------------------------------

Output ottenuto:
['tto']

Risultato Test: NEGATIVO

******************************

Test funzione: trovaParole2 

Input funzione: 'I_Malavoglia_50.txt', 2 

Output atteso:
['addii', 'affrettati', 'ammarrata', 'apparecchiare', 'arricchirsi', 'arricchisci', 'arricchito', 'donnicciuole', 'fazzoletto', 'gruzzoletto', 'parommella', 'parrocchia', 'ricchezze', 'scappellotto', 'soddisfatti', 'soggetto', 'villaggio', 'vorrebbe', 'zuppidda']

----- print interne funzione -----

----------------------------------

Output ottenuto:
['bbe', 'cchia', 'cchiare', 'cchirsi', 'cchisci', 'cchito', 'cciuole', 'dda', 'ggio', 'ii', 'lla', 'rrata', 'ttati', 'tti', 'tto', 'zze']

Risultato Test: NEGATIVO

******************************

Test funzione: trovaParole2 

Input funzione: 'I_Malavoglia.txt', 3 

Output atteso:
['abboccherebb

0

#### Esercizio 3:
Calcolare quante volte in un file compare una sequenza di 3 parole consecutive con la proprietà che tutte le parole cominciano con la stessa lettera e finiscono con la stessa lettera (lettera iniziale e finale possono essere diverse). 

**Nota:** Si assuma che le parole di ciascuna sequenza contengano almeno due lettere e non si considerino sequenze sovrapposte (ad esempio, nella stringa 'are asce asole ace ave' c'è comunque una sola sequenza, perchè 'asole' deve essere considerato parte di una sola sequenza)

In [72]:
from tester import tester_fun

import re

def conta(file):
    fin=open(file,'r',encoding='UTF-8')
    testo=fin.read()
    pat=r'\b(\w)\w*(\w)\b\W*\b\1\w*\2\b\W*\b\1\w*\2\b'
    r=re.findall(pat,testo,re.IGNORECASE)
    fin.close()
    return len(r)
counter_test_positivi = 0

counter_test_positivi += tester_fun(conta,['file4.txt'],1)
counter_test_positivi += tester_fun(conta,['file5.txt'],2)
counter_test_positivi += tester_fun(conta,['file6.txt'],3)

print('La funzione',conta.__name__,'ha superato',counter_test_positivi,'test')


Test funzione: conta 

Input funzione: 'file4.txt' 

Output atteso:
1

----- print interne funzione -----

----------------------------------

Output ottenuto:
1

Risultato Test: POSITIVO

******************************

Test funzione: conta 

Input funzione: 'file5.txt' 

Output atteso:
2

----- print interne funzione -----

----------------------------------

Output ottenuto:
2

Risultato Test: POSITIVO

******************************

Test funzione: conta 

Input funzione: 'file6.txt' 

Output atteso:
3

----- print interne funzione -----

----------------------------------

Output ottenuto:
3

Risultato Test: POSITIVO

******************************

La funzione conta ha superato 3 test


#### Esercizio 4:
Il programma che abbiamo visto per l'estrazione dei codici fiscali risolve il problema, ma non è completamente preciso. Per essere precisi, bisognerebbe verificare che il numero sia coerente con il mese (Aprile, Giugno, Settembre e Novembre) hanno 30 giorni e Febbraio ne ha 28 (o 29). Provate a completare quell'esercizio introducendo questi controlli addizionali. Assumete che un anno sia bisestile se è un multiplo di 4 (semplificando così leggermente il calcolo).

In [74]:
from tester import tester_fun

import re

def trovaCF2(file):
    fin=open(file,'r',encoding='UTF-8') 
    testo=fin.read()
    fin.close()
    pat=r'[A-Z]{3} ?[A-Z]{3} ?(\d){2}([A-Z])(\d){2} ?[A-Z]\d{3}[A-Z]'
    r=re.findall(pat,testo)
    uom=[]
    don=[]
    mesi = 'ABCDEHLMPRST'
    for m in r:
        tut=m.group(0)
        mese=m.group(2)
        giorno=int(m.group(3))
        anno=int(m.grup(1))
        
        if (1<= giorno <=31) or (41<=girono<=71) and mese in mesi:
            if giorno<=31:
                uom.append(tut)
            else:
                don.append(tut)
    uom=uom.sort()
    
counter_test_positivi = 0

tester_fun(trovaCF2, ['testoDateCF.txt'],(['CCCMRC17A11H501W', 'CCDMRC55P13H501W'], ['CCCMRC00M57H501W', 'CCCMRC96E41H501W']))
tester_fun(trovaCF2, ['testoDateCF2.txt'],(['CCDMRC80B29H501W'], ['CCCMRC71T57H501W', 'CCCSRC74D51H501W']))
tester_fun(trovaCF2, ['testoDateCF3.txt'],(['CCCMRC57D30H501W', 'CCDMRC80B29H501W'], ['CCCSRC57T51H501W']))

print('La funzione',trovaCF2.__name__,'ha superato',counter_test_positivi,'test')
   

Test funzione: trovaCF2 

Input funzione: 'testoDateCF.txt' 

Output atteso:
(['CCCMRC17A11H501W', 'CCDMRC55P13H501W'], ['CCCMRC00M57H501W', 'CCCMRC96E41H501W'])

----- print interne funzione -----

----------------------------------

La funzione ha lanciato una eccezione durante il test

Traceback (most recent call last):
  File "c:\Users\campe\Desktop\Vittorio UNI\Programazione\15EspressioniRegolari-20231201T175217Z-001\15EspressioniRegolari\tester.py", line 97, in helper_tester_function
    returned_value = function(*input_data)
                     ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\campe\AppData\Local\Temp\ipykernel_21724\2957077108.py", line 15, in trovaCF2
    mese=m.group(2)
         ^^^^^^^
AttributeError: 'tuple' object has no attribute 'group'

Risultato Test: NEGATIVO

******************************

Test funzione: trovaCF2 

Input funzione: 'testoDateCF2.txt' 

Output atteso:
(['CCDMRC80B29H501W'], ['CCCMRC71T57H501W', 'CCCSRC74D51H501W'])

----- print interne funzione